In [ ]:
%%bash
touch './src/sim_files/test.npy'

In [ ]:
# save numpy array as npy file
import numpy as np
# save to npy file
np.save('./src/sim_files/data.npy', data)

In [ ]:
import numpy as np
dataLoaded = np.load('./src/sim_files/data.npy')

# AES

In [ ]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
print("Our send msg looks like this: "+ str(msg))
target.simpleserial_write('p', msg)

In [ ]:
received_msg = target.simpleserial_read('r', 16)
print("Our received msg looks like this: "+ str(received_msg))

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET='AVRCRYPTOLIB'
SS_VER='SS_VER_1_1'

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../chipwhisperer/hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
cw.program_target(scope, prog, "../../chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

In [ ]:
ktp = cw.ktp.Basic()
key, text = ktp.next()
target.set_key(key)
target.simpleserial_write('p', text)
response = target.simpleserial_read('r', 16)
print('Text in : '+str(text))
print('Text out : '+str(response))

In [ ]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()
target.set_key(key)

N = 2500
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 
    
assert(len(trace_array) == 2500)
print("✔️ OK to continue!")

# RSA

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'
#SS_VER='SS_VER_2_0'

In [2]:
%run "Setup_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400
INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../chipwhisperer/hardware/victims/firmware/simpleserial-rsa
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-rsa-CWLITEXMEGA.hex
rm -f -- simpleserial-rsa-CWLITEXMEGA.eep
rm -f -- simpleserial-rsa-CWLITEXMEGA.cof
rm -f -- simpleserial-rsa-CWLITEXMEGA.elf
rm -f -- simpleserial-rsa-CWLITEXMEGA.map
rm -f -- simpleserial-rsa-CWLITEXMEGA.sym
rm -f -- simpleserial-rsa-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-rsa.s simpleserial-rsa-xmega.s simpleserial-rsa-arm.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s rsa_basic.s bigint.s
rm -f -- simpleserial-rsa.d simpleserial-rsa-xmega.d simpleserial-rsa-arm.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d rsa_basic.d bigint.d
rm -f -- simpleserial-rsa.i simpleserial-rsa-xmega.i simpleserial-rsa-arm.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i rsa_basic.i bigint.i
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free so

simpleserial-rsa.c: In function â€˜mainâ€™:
simpleserial-rsa.c:45:34: error: â€˜sig_chunk_1â€™ undeclared (first use in this function)
     simpleserial_addcmd('1', 0,  sig_chunk_1);
                                  ^
simpleserial-rsa.c:45:34: note: each undeclared identifier is reported only once for each function it appears in
simpleserial-rsa.c:46:34: error: â€˜sig_chunk_2â€™ undeclared (first use in this function)
     simpleserial_addcmd('2', 0,  sig_chunk_2);
                                  ^
make: *** [.././Makefile.inc:469: objdir/simpleserial-rsa.o] Error 1


CalledProcessError: Command 'b'cd ../../chipwhisperer/hardware/victims/firmware/simpleserial-rsa\nmake PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3\n'' returned non-zero exit status 2.

In [ ]:
# cw.program_target(scope, prog, '../../chipwhisperer/hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CW303.hex')
cw.program_target(scope, prog, '../../chipwhisperer/hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex')

In [ ]:
# some CW setup for RSA
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 24000 # to capture enough data we will take 24000 measurements per trace

%matplotlib notebook
import matplotlib.pylab as plt

text1 = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0x0B, 0x0F])
# 0xB0F = 0b1011 0000 1111
scope.arm()
target.simpleserial_write('p', text1)
ret = scope.capture()
if ret:
    trace1 = 'NONE'
target.simpleserial_wait_ack()
trace1 = scope.get_last_trace()

plt.plot(trace1, 'b')